# [프로젝트3] 고객 상담 문의 유형 분류를 위해 RNN 기반 모델 적용하기

---


## 프로젝트 목표
---
- 딥러닝 모델에서 처리 가능하도록 데이터 전처리
- RNN 기반 모델 중 하나인 LSTM을 딥러닝 프레임워크로 구성
- LSTM을 통하여 문의 유형 분류 모델 학습 및 평가


## 프로젝트 목차
---

1. **데이터 전처리:** 프로젝트 1에서 가져온 데이터를 딥러닝 프레임워크에서 쓸 수 있게 전처리합니다.

2. **LSTM 모델 구성:** RNN 기반 모델 중 하나인 LSTM을 torch 기반으로 구성합니다.

3. **LSTM을 통한 문의 유형 분류 문제:** LSTM을 통하여 문의 유형 분류 문제를 해결할 수 있도록 LSTM을 학습 및 평가합니다.


## 프로젝트 개요
---

프로젝트 1에서 불러오고 자연어 전처리한 데이터를 바탕으로 본격적으로 분류 문제를 딥러닝 모델로 해결해 보고자 합니다.

이를 위하여 데이터를 딥러닝 모델의 입력값이 될 수 있도록 전처리를 하고, 딥러닝 모델 중 하나인 LSTM을 `Pytorch` 기반으로 구성합니다.

구성한 모델을 고객 상담 기록 데이터로 학습하여 문의 유형 분류 문제를 풀 수 있도록 LSTM을 학습 및 평가합니다.

## 1. 데이터 전처리

---

### 1.1. 라이브러리 및 데이터 불러오기


프로젝트 1에서 사용한 데이터와 모델 학습을 위해 필요한 라이브러리를 불러옵니다. 

In [1]:
!pip install torch torchtext==0.11.0

     |████████████████████████████████| 8.0 MB 20.1 MB/s eta 0:00:01
     |███████████▌                    | 315.9 MB 125.2 MB/s eta 0:00:05

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |█████████████████████████       | 691.1 MB 118.2 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 2.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import re
from konlpy.tag import Okt

import random
import numpy as np
import torch
import torchtext

In [3]:
data = pd.read_csv('./01_data.csv', encoding='cp949')
texts = data['메모'].tolist() # 자연어 데이터를 리스트 형식으로 변환합니다
label_list = data['상담유형3_GT'].unique().tolist()
labels = data['상담유형3_GT'].tolist()

In [4]:
def cleaning(text):
    # 정제: 한글, 공백 제외한 문자 제거
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', text)
    return text

In [5]:
texts_clean = []
for i in range(len(texts)):
    text_clean = cleaning(texts[i])
    texts_clean.append(text_clean)

학습 데이터와 테스트 데이터를 구분합니다.

In [6]:
num_train = int(0.8*len(texts_clean))

texts_labels = list(zip(texts_clean,labels))
random.shuffle(texts_labels)
texts_clean, labels = zip(*texts_labels)

train_texts = texts_clean[:num_train]
train_labels = labels[:num_train]

test_texts = texts_clean[num_train:]
test_labels = labels[num_train:]

In [7]:
train_data = pd.DataFrame({'text': train_texts,
                          'label': train_labels})
test_data = pd.DataFrame({'text': test_texts,
                          'label': test_labels})

In [8]:
train_data.to_csv('./train_data.csv',index=False)
test_data.to_csv('./test_data.csv',index=False)

### 1.2. 데이터 전처리
---

List 형태로 저장되어 있는 데이터와 라벨을 torch 모델에 적용할 수 있도록 전처리합니다. 이때, torchtext 라이브러리를 사용합니다.

In [9]:
tokenizer = Okt()

TEXT = torchtext.legacy.data.Field(tokenize=tokenizer.morphs,
                 include_lengths=True)

LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)

fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}

train, validation, test 데이터를 구분지어 만듭니다.

In [10]:
train_data, test_data = torchtext.legacy.data.TabularDataset.splits(
                            path = './',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)

In [11]:
train_data, valid_data = train_data.split()

자연어 데이터를 컴퓨터로 표현하기 위한 임베딩 벡터를 가져옵니다. 본 프로젝트에서는 한국어 임베딩이 있는 FastText 서브 워드 임베딩을 사용합니다.

In [12]:
TEXT.build_vocab(train_data,
                 max_size = 10000,
                 vectors = 'fasttext.simple.300d',
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/wiki.simple.vec: 293MB [00:40, 7.18MB/s]                               
100%|██████████| 111051/111051 [00:08<00:00, 13212.93it/s]


In [13]:
batch_size = 64

device = torch.device('cpu')

train_iterator, valid_iterator, test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = batch_size,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

In [14]:
next(iter(train_iterator)).text

(tensor([[  73,  130,  182,  400,   10,  122,   10,  125,  130,  122,   11,  800,
          4092,  829,   11,  136,   89,   11,   66,  125,  403,  163, 1041,   11,
           179,  627,   57, 2295,  237,  436,  123,  479,  459,   41,  108,   94,
           281,  400,   36,   73,  415,   73,   73,  106,  146,   10,   11,   11,
          1011,   80,   11,   94,  400, 1823,  229,   41,  321,   10,  181,  788,
            73,   94,   41,  305],
         [  62, 1205,  690,  178,  111,  348,   13,   51, 1205,   51,   98,   88,
           119,  124,   98,  172, 1325,   98,   65,   56,  469,   11,   48,   98,
           270,   17,   76, 3925,  386,   95,  163,  232,  207,   30,   13,   79,
            50,  179,   73,  491,   10,  491,  491,   17, 1010,   27,   42,   98,
          3441,   56,  148,   79,   43,  109,   22,  193,   72,  119,   27,   43,
           491,   79,   17,  367],
         [ 203,   51,  875,   26,   13,  120,   11,  511, 3455,   32,   56,  216,
           120,   22,   95, 

In [15]:
print(LABEL.vocab.itos, len(LABEL.vocab.stoi))

['발수신불가', '통품상담성', '기업형메시지(금융권인증SMS)', '초기접속불가', '속도저하', '호단절', '단문메세지(SMS)', '이설요청/철거요청', '기타 Application', '데이터 사용중 끊김', '음질불량', 'T제공 어플', 'MMS', '수신불가', '3G천이', '발신불가', '카카오', '통화중대기'] 18


## 2. LSTM 모델 구성

---

LSTM 분류 모델을 torch로 구성합니다.


In [16]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [17]:
class LSTMClassifier(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, pad_idx):
        super(LSTMClassifier, self).__init__()
        # 단어 임베딩
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        # LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # 분류자 (classifier)
        self.fc = nn.Linear(hidden_dim, tagset_size)

    def forward(self, text, text_length):
        embeds = self.word_embeddings(text)
        packed_embeds = nn.utils.rnn.pack_padded_sequence(embeds, text_length)
        _, (hidden, cell) = self.lstm(packed_embeds)
        logits = self.fc(hidden.squeeze(0))
        scores = F.log_softmax(logits, dim=1)
        return scores

## 3. LSTM을 통한 문의 유형 분류 문제

---

### 3.1 하이퍼파라미터 설정


In [18]:
VOCAB_SIZE = len(TEXT.vocab) # 단어 개수
EMBEDDING_DIM = 300 # 임베딩 차원
HIDDEN_DIM = 256 # 은닉 상태 차원
TARGET_SIZE = len(LABEL.vocab.stoi) # 라벨 클래스 개수
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # 패딩 인덱스


### 3.2 모델 학습

모델, 손실 함수 (loss function), 옵티마이저 (optimizer) 설정

In [19]:
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TARGET_SIZE, PAD_IDX)
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

임베딩을 사전 학습된 FastText 임베딩으로 덮어씌웁니다.

In [20]:
pretrained_embeddings = TEXT.vocab.vectors
model.word_embeddings.weight.data.copy_(pretrained_embeddings) 

tensor([[-2.4905, -1.5179, -0.0916,  ...,  1.5286,  0.2033, -0.4287],
        [-0.7065,  0.2575, -0.9770,  ..., -0.9159,  0.1888,  1.1423],
        [ 0.8592, -0.7363,  0.4632,  ..., -0.6997,  1.6823, -0.6281],
        ...,
        [-0.0139, -2.4191,  1.7068,  ...,  1.1082,  0.2201, -1.2869],
        [ 1.2630,  0.4783,  0.7196,  ...,  1.5432,  1.5518,  0.8923],
        [-1.3336,  2.5820,  0.6197,  ..., -0.2017,  0.9644, -0.0027]])

사전에 정의되지 않은 단어에 대한 토큰인 `<UNK>`와 빈 칸을 위한 토큰인 `<PAD>`를 0 벡터로 설정합니다.

In [21]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
print(UNK_IDX, PAD_IDX)

model.word_embeddings.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.word_embeddings.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.word_embeddings.weight.data)

0 1
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.8592, -0.7363,  0.4632,  ..., -0.6997,  1.6823, -0.6281],
        ...,
        [-0.0139, -2.4191,  1.7068,  ...,  1.1082,  0.2201, -1.2869],
        [ 1.2630,  0.4783,  0.7196,  ...,  1.5432,  1.5518,  0.8923],
        [-1.3336,  2.5820,  0.6197,  ..., -0.2017,  0.9644, -0.0027]])


### [TODO] 모델 성능 평가를 위하여 정확도를 예측하는 함수를 작성해 주세요.

In [22]:
def accuracy(prediction, label):
    '''
    input
    prediction: 배치에 대하여 각 클래스 별 log-softmax 값, shape (batch_size, num_classes)
    label: 배치에 대하여 클래스 라벨, shape (batch_size,)
    '''
    prediction_argmax = prediction.max(dim=-1)[1] # prediction 값을 통하여 데이터 별 예측 클래스 추출
    correct = (prediction_argmax == label).float() # 각 데이터 별로 예측 값이 실제 라벨을 맞췄는지 확인
    acc = correct.sum() / len(correct) # 배치에 대한 정확도
    return acc

훈련 함수를 정의합니다.

In [23]:
def train(model, iterator, optimizer, loss_function):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        text, text_length = batch.text
        if 0. in text_length:
            continue
        predictions = model(text, text_length)
        
        loss = loss_function(predictions, batch.label)
        acc = accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### [TODO] 위의 train 함수를 참고하여 모델을 평가하는 함수를 작성하세요.

In [24]:
def evaluate(model, iterator, loss_function):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_length = batch.text # batch에서 텍스트와 텍스트 길이 구분
            if 0. in text_length:
                continue
            predictions = model(text, text_length) # LSTM 모델에 데이터 주입 
            
            loss = loss_function(predictions, batch.label) # 로스 계산
            acc = accuracy(predictions, batch.label) # 정확도 계산
            
            epoch_loss += loss.item() # 전체 로스 계산을 위한 저장
            epoch_acc += acc.item() # 전체 정확도 계산을 위한 저장
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

학습을 진행합니다.

In [25]:
NUM_EPOCHS = 10
best_valid_loss = float('inf')

In [26]:
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_function)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_function)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm-best.pt')
        
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Train Loss: 2.510 | Train Acc: 21.37%
	 Val. Loss: 2.147 |  Val. Acc: 25.55%
Epoch: 02
	Train Loss: 1.919 | Train Acc: 35.65%
	 Val. Loss: 1.862 |  Val. Acc: 35.58%
Epoch: 03
	Train Loss: 1.611 | Train Acc: 45.76%
	 Val. Loss: 1.709 |  Val. Acc: 40.90%
Epoch: 04
	Train Loss: 1.330 | Train Acc: 54.26%
	 Val. Loss: 1.552 |  Val. Acc: 45.89%
Epoch: 05
	Train Loss: 1.069 | Train Acc: 62.83%
	 Val. Loss: 1.454 |  Val. Acc: 49.62%
Epoch: 06
	Train Loss: 0.891 | Train Acc: 69.16%
	 Val. Loss: 1.524 |  Val. Acc: 49.04%
Epoch: 07
	Train Loss: 0.734 | Train Acc: 74.38%
	 Val. Loss: 1.406 |  Val. Acc: 53.02%
Epoch: 08
	Train Loss: 0.561 | Train Acc: 80.10%
	 Val. Loss: 1.476 |  Val. Acc: 49.29%
Epoch: 09
	Train Loss: 0.439 | Train Acc: 84.73%
	 Val. Loss: 1.359 |  Val. Acc: 58.47%
Epoch: 10
	Train Loss: 0.319 | Train Acc: 89.35%
	 Val. Loss: 1.352 |  Val. Acc: 59.54%


테스트 데이터에 대하여 평가를 진행합니다.

In [27]:
model.load_state_dict(torch.load('lstm-best.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_function)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.380 | Test Acc: 57.91%
